# Context-sensitive Spelling Correction

The goal of the assignment is to implement context-sensitive spelling correction. The input of the code will be a set of text lines and the output will be the same lines with spelling mistakes fixed.

Submit the solution of the assignment to Moodle as a link to your GitHub repository containing this notebook.

Useful links:
- [Norvig's solution](https://norvig.com/spell-correct.html)
- [Norvig's dataset](https://norvig.com/big.txt)
- [Ngrams data](https://www.ngrams.info/download_coca.asp)

Grading:
- 60 points - Implement spelling correction
- 20 points - Justify your decisions
- 20 points - Evaluate on a test set


## Implement context-sensitive spelling correction

Your task is to implement context-sensitive spelling corrector using N-gram language model. The idea is to compute conditional probabilities of possible correction options. For example, the phrase "dking sport" should be fixed as "doing sport" not "dying sport", while "dking species" -- as "dying species".

The best way to start is to analyze [Norvig's solution](https://norvig.com/spell-correct.html) and [N-gram Language Models](https://web.stanford.edu/~jurafsky/slp3/3.pdf).

You may also want to implement:
- spell-checking for a concrete language - Russian, Tatar, etc. - any one you know, such that the solution accounts for language specifics,
- some recent (or not very recent) paper on this topic,
- solution which takes into account keyboard layout and associated misspellings,
- efficiency improvement to make the solution faster,
- any other idea of yours to improve the Norvig’s solution.

IMPORTANT:  
Your project should not be a mere code copy-paste from somewhere. You must provide:
- Your implementation
- Analysis of why the implemented approach is suggested
- Improvements of the original approach that you have chosen to implement

In [88]:
def open_data_grams(path):
    "Load English txt"
    data = open(path).read()
    rows = data.split('\n')
    grams = {}
    for row in rows:
        key = " ".join([word for word in row.split('\t')[1:]])
        v = int(row.split('\t')[0])
        grams[key] = v
    return grams

BI_GRAMS = open_data_grams('datasets/bigrams.txt')
FIVE_GRAMS = open_data_grams('datasets/fivegrams.txt')

## Norvig's solution

In [101]:
class NorvigSpellChecker():
    def __init__(self, letters:str) -> None:
        self.letters = letters
        pass

    def add_grams(self, grams:dict):  
        "Append a dictionary of GRAMS with their frequencies"  
        self.GRAMS = grams
        self.n_grams = len(list(self.GRAMS.keys())[0].split(' '))

    def P(self, gram:str): 
        "Probability of `gram`."
        if 0 <= self.GRAMS[gram] <= 1:
            return self.GRAMS[gram]
        else: 
            return self.GRAMS[gram] / sum(self.GRAMS.values())

    def correction(self, gram:str): 
        "Most probable spelling correction for gram."
        return max(self.candidates(gram), key=self.P)

    def candidates(self, gram:str): 
        "Generate possible spelling corrections for gram."
        return (self.known([gram]) or self.known(self.edits1(gram)) or self.known(self.edits2(gram)) or [gram])

    def known(self, grams:str): 
        "The subset of `grams` that appear in the dictionary of GRAMS."
        return set(g for g in grams if g in self.GRAMS)

    def edits1(self, gram:str):
        "All edits that are one edit away from `gram`."
        splits     = [(gram[:i], gram[i:])    for i in range(len(gram) + 1)]
        deletes    = [L + R[1:]               for L, R in splits if R]
        transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
        replaces   = [L + c + R[1:]           for L, R in splits if R for c in self.letters]
        inserts    = [L + c + R               for L, R in splits for c in self.letters]
        return set(deletes + transposes + replaces + inserts)

    def edits2(self, gram:str): 
        "All edits that are two edits away from `gram`."
        return (e2 for e1 in self.edits1(gram) for e2 in self.edits1(e1))

#### 1) Test Norvig's solution on bigram english data

In [102]:
bi_sc = NorvigSpellChecker('abcdefghijklmnopqrstuvwxyz')
bi_sc.add_grams(BI_GRAMS)
print("Bigram data")
print(f"The frequency of gram `a aaa`: {round(bi_sc.P('a aaa'), 9)}")
print(f"The number of grams: {bi_sc.n_grams}")

Bigram data
The frequency of gram `a aaa`: 1.08e-07
The number of grams: 2


In [142]:
bi_sc.correction('a badysiter')

'a babysitter'

#### 2) Test Norvig's solution on fivegram english data

In [103]:
list(FIVE_GRAMS.keys())[0]

'a babe in the woods'

In [104]:
fv_sc = NorvigSpellChecker('abcdefghijklmnopqrstuvwxyz')
fv_sc.add_grams(FIVE_GRAMS)
print("Fivegram data")
print(f"The frequency of gram `a babe in the woods`: {round(fv_sc.P('a babe in the woods'), 9)}")
print(f"The number of grams: {fv_sc.n_grams}")

Fivegram data
The frequency of gram `a babe in the woods`: 9.73e-07
The number of grams: 5


In [114]:
fv_sc.candidates('married to the same oman')

{'married to the same man', 'married to the same woman'}

## Improvement

First of all, to improve Norvig's solution that was decided to adapted it for the Russian language, as it my native language and I familiar with it syntax and semantic, and, also to add an opportunity to apply keyboard layout for for spelling correction.

### 1) Spelling corrector for English language with an opportunity to apply keyboard layout.

In [106]:
class ENSpellChecker(NorvigSpellChecker):
    
    def __init__(self) -> None:
        self.keyboard = ['qwertyuiop', 'asdfghjkl', 'zxcvbnm', ' ']
        super().__init__('abcdefghijklmnopqrstuvwxyz ')

    def find_keybord_distance(self, leter1:str, leter2:str):
        "Calulate distance between two letters in word according to their keyboard layout"
        r1, r2, c1, c2 = -1, -1, -1, -1
        if leter1 == ' ' or leter2 == ' ':
            for i, row in enumerate(self.keyboard, start=1):
                if leter1 in row or leter2 in row:
                    return 4 - i
        for i, row in enumerate(self.keyboard):
            if leter1 in row:
                r1 = i
                c1 = row.index(leter1)
            if leter2 in row:
                r2 = i
                c2 = row.index(leter2)
        
        dist = max(abs(r1 - r2), abs(c1 - c2))
        return dist

    def keyboard_P(self, candidate_gram:str, gram:str):
        "Calculate the cost of `gram` with keyboard layout"
        keyboard_p = 0
        for i, j in zip(candidate_gram, gram):
            keyboard_p += self.find_keybord_distance(i, j)
        return keyboard_p
    
    def is_correct(self, gram:str):
        "Check if gram is correct or not"
        return gram in self.GRAMS.keys()

    def correction(self, gram:str, with_keyboard_layout:bool=False):
        "Most probable spelling correction for gram."
        gram = gram.lower()
        if with_keyboard_layout:
            top3 = sorted(self.candidates(gram), key=self.P, reverse=True)[:3]
            return min(top3, key=lambda candidate: self.keyboard_P(candidate, gram))
        else: 
            return max(self.candidates(gram), key=self.P)

* Test on bigram data

In [109]:
en_bi_sc = ENSpellChecker()
en_bi_sc.add_grams(BI_GRAMS)
print(f"The `a coetect` is in the vocabulary: {en_bi_sc.is_correct('a coetect')}")
print(f"With keyboard layout the correction of `a coetect`: {en_bi_sc.correction('a coetect', with_keyboard_layout=True)}")
print(f"Without keyboard layout the correction of `a coetect`: {en_bi_sc.correction('a coetect', with_keyboard_layout=False)}")

The `a coetect` is in the vocabulary: False
With keyboard layout the correction of `a coetect`: a correct
Without keyboard layout the correction of `a coetect`: a context


In [110]:
bigram = 'lghts shune'
print(f"The `{bigram}` is in the vocabulary: {en_bi_sc.is_correct(bigram)}")
print(f"With keyboard layout the correction of `{bigram}`: {en_bi_sc.correction(bigram, with_keyboard_layout=True)}")
print(f"Without keyboard layout the correction of `{bigram}`: {en_bi_sc.correction(bigram, with_keyboard_layout=False)}")

The `lghts shune` is in the vocabulary: False
With keyboard layout the correction of `lghts shune`: lights shine
Without keyboard layout the correction of `lghts shune`: lights shone


* Test on fivegram data

In [116]:
en_fv_sc = ENSpellChecker()
en_fv_sc.add_grams(FIVE_GRAMS)
fivegram = 'married to the same oman'
print(f"The `{fivegram}` is in the vocabulary: {en_fv_sc.is_correct(fivegram)}")
print(f"With keyboard layout the correction of `{fivegram}`: {en_fv_sc.correction(fivegram, with_keyboard_layout=True)}")
print(f"Without keyboard layout the correction of `{fivegram}`: {en_fv_sc.correction(fivegram, with_keyboard_layout=False)}")

The `married to the same oman` is in the vocabulary: False
With keyboard layout the correction of `married to the same oman`: married to the same man
Without keyboard layout the correction of `married to the same oman`: married to the same woman


### 2) Spelling corrector for Russian language with an opportunity to apply keyboard layout.

In [151]:
import re

class RUSpellChecker(NorvigSpellChecker):
    def __init__(self) -> None:
        self.keyboard = ['йцукенгшщзхъ', 'фывапролджэ', 'ячсмитьбю', ' '] 
        super().__init__('абвгдеёжзийклмнопрстуфхцчшщъыьэюя ')
    
    def is_num(sel, gram:str):
        "Check if `_num_` in the gram"
        return re.findall('\d+', gram)
    
    def find_keybord_distance(self, leter1:str, leter2:str):
        "Calulate distance between two letters in word according to their keyboard layout"
        r1, r2, c1, c2 = -1, -1, -1, -1
        if leter1 == ' ' or leter2 == ' ':
            for i, row in enumerate(self.keyboard, start=1):
                if leter1 in row or leter2 in row:
                    return 4 - i
        for i, row in enumerate(self.keyboard):
            if leter1 in row:
                r1 = i
                c1 = row.index(leter1)
            if leter2 in row:
                r2 = i
                c2 = row.index(leter2)
        
        dist = max(abs(r1 - r2), abs(c1 - c2))
        return dist

    def keyboard_P(self, candidate_gram:str, gram:str):
        "Calculate the cost of `gram` with keyboard layout"
        keyboard_p = 0
        for i, j in zip(candidate_gram, gram):
            keyboard_p += self.find_keybord_distance(i, j)
        return keyboard_p
    
    def correction(self, gram:str, with_keyboard_layout:bool=False): 
        "Most probable spelling correction for gram."
        gram = gram.lower()
        if self.is_num(gram):
            new_gram = re.sub('\d+', '_num_', gram)
            best = self.correct(new_gram, with_keyboard_layout)
            correct = re.sub('_num_', self.is_num(gram)[0], best)
            return correct
        else:
            return self.correct(gram, with_keyboard_layout)
        
    def correct(self, gram:str, with_keyboard_layout:bool=False):
        "Most probable spelling correction for gram."
        gram = gram.lower()
        if with_keyboard_layout:
            top3 = sorted(self.candidates(gram), key=self.P, reverse=True)[:3]
            return min(top3, key=lambda candidate: self.keyboard_P(candidate, gram))
        else: 
            return max(self.candidates(gram), key=self.P)

In [152]:
def open_rus_data_grams(path):
    "Load Russian txt"
    data = open(path, encoding="UTF-8").read()
    rows = data.split('\n')
    grams = {}
    for row in rows:
        if not row:
            continue
        key = " ".join([word for word in row.split('\t')[:-1]])
        v = float(row.split('\t')[-1])
        grams[key] = v
    return grams

RUS_GRAMS = open_rus_data_grams('datasets/rus_bigrams.txt')

In [153]:
data = list(RUS_GRAMS.keys())
print(data[:10])

['об этом', '_num_ года', 'может быть', '_num_ году', '_num_ _num_', 'у него', 'у нас', 'у меня', 'потому что', 'ничего не']


In [154]:
ru_sc = RUSpellChecker()
ru_sc.add_grams(RUS_GRAMS)
print(f"Testing Russian Corrector on `вот этоо`, the correction: {ru_sc.correction('вот этоо')}")
print(f"Testing Russian Corrector on `2457 гора` with numeric elements, the correction: {ru_sc.correction('2457 гора')}")

Testing Russian Corrector on `вот этоо`, the correction: вот это
Testing Russian Corrector on `2457 гора` with numeric elements, the correction: 2457 года


### Applying for completed sentences

In [137]:
def sent_correction(corrector:NorvigSpellChecker, sent:str, with_keyboard_layout:bool=False):
    "Correcting whole sentences"
    n_grams = corrector.n_grams
    i = 0
    while i + n_grams <= len(sent.split(' ')):
        gram = sent.split(' ')[i:i + n_grams]
        gram = " ".join(str(element) for element in gram)
        i += 1
        new_gram = corrector.correction(gram, with_keyboard_layout=with_keyboard_layout)
        sent = sent.replace(gram, new_gram)
    return sent

print(f"Correction of the sentence `i aggree wiht you`: {sent_correction(en_bi_sc, 'i aggree wiht you')}")
print(f"Correction of the sentence `у его нечего нет`: {sent_correction(ru_sc, 'у его нечего нет')}")

Correction of the sentence `i aggree wiht you`: i agree with you
Correction of the sentence `у его нечего нет`: у него ничего нет


In [166]:
sent = "Once apon a time fot them girl who made a wisch"
print(f"Correction of the sentence by bigram model: {sent_correction(en_bi_sc, sent)}")
print(f"Correction of the sentence by bigram model: {sent_correction(en_bi_sc, sent, True)}")
# print(f"Correction of the sentence by fivegram model: {sent_correction(en_fv_sc, sent)}")

Correction of the sentence by bigram model: once upon a time for the girl who made a witch
Correction of the sentence by bigram model: once upon a time for them girls who made a witch


## Justify your decisions

Write down justificaitons for your implementation choices. For example, these choices could be:
- Which ngram dataset to use
- Which weights to assign for edit1, edit2 or absent words probabilities
- Beam search parameters
- etc.

During the process, it was learned that the dataset of five grams was too small and many combinations seemed unfamiliar to the model unlike the bigrams dataset. Furthermore, fivegrams dataset would be more successful on long sentences.

## Evaluate on a test set

Your task is to generate a test set and evaluate your work. You may vary the noise probability to generate different datasets with varying compexity. Compare your solution to the Norvig's corrector, and report the accuracies.

#### 1) Evaluating of english corrector 

In [139]:
# The test sets were generated by CHAT GPT

english_test = [
    "I love too eat ice cream", 
    "She is very quite in class",
    "The whether is changing",
    "He bougth a pare of shoes",
    "The cat is laying im the sun",
    "She can heae the music from the next room",
    "I would like some mote cofee",
    "The restaran is loosing money",
    "He asjed for a piece of pir"
    ]

english_true = [
    "I love to eat ice cream", 
    "She is very quiet in class",
    "The weather is changing",
    "He bought a pair of shoes",
    "The cat is lying in the sun",
    "She can hear the music from the next room",
    "I would like some more coffee",
    "The restaurant is losing money",
    "He asked for a piece of pie"
]

* Without keyboard layout (Norvig's solution)

In [140]:
for sent, true in zip(english_test, english_true):
    sent = sent.lower()
    true = true.lower()
    correction = sent_correction(en_bi_sc, sent)
    if correction == true:
        print(f"{correction} == {true}")
    else:
        print(f"{correction} != {true}")

i love to eat ice cream == i love to eat ice cream
she is very quiet in class == she is very quiet in class
them whether it changing != the weather is changing
he bought a part of shoes != he bought a pair of shoes
the cat is laying in the sun != the cat is lying in the sun
she can hear the music from the next room == she can hear the music from the next room
i would like some more coffee == i would like some more coffee
the restaurant is losing money == the restaurant is losing money
he asked for a piece of air != he asked for a piece of pie


The result: english corrector without keyboard layout corrected 5 out of 9 sentences.

* With keyboard layout

In [141]:
for sent, true in zip(english_test, english_true):
    sent = sent.lower()
    true = true.lower()
    correction = sent_correction(en_bi_sc, sent, with_keyboard_layout=True)
    if correction == true:
        print(f"{correction} == {true}")
    else:
        print(f"{correction} != {true}")

i love too eat ice cream != i love to eat ice cream
she is very quiet in class == she is very quiet in class
then whether a changing != the weather is changing
he bought a page of shoes != he bought a pair of shoes
the cat is laying in the sun != the cat is lying in the sun
she can hear the music from the next room == she can hear the music from the next room
i would like some more coffee == i would like some more coffee
the restart it looking more != the restaurant is losing money
he asked for a piece of pie == he asked for a piece of pie


The result: english corrector with keyboard layout corrected 4 out of 9 sentences.

In [159]:
sent = "To impove the solusion they was decided too add the Russia languag as ir my native language and I familar wiht it suntax and semantik and to add a opportunety to apply key bord layout for colection."
print(f"Correction of the sentence by bigram model: {sent_correction(en_bi_sc, sent)}")
print(f"Correction of the sentence by bigram model: {sent_correction(en_bi_sc, sent, True)}")

Correction of the sentence by bigram model: to improve the solution they was decided to add the russian language as if my native language and a familiar with it sunday and semantic and to add an opportunity to apply any word about for collection
Correction of the sentence by bigram model: to impose the solution they was decided to add the russian language as i my native language and a familiar with it sunday and semantic and to add as opportunity to apply my word about for collection


#### 1) Evaluating of russian corrector 

In [218]:
russian_test = [
    "Я купили билет на рынке",
    "Мы пойдем на балл",
    "Она выгледит очень хорошо",
    "Вчера был тижeлый день на работе",
    "Он учитса в университет",
    "Эта требует вркмени",
    "Я встречаюс с ней ужe два нода",
    "Мы собираемса на море в июне",
    "Пулец вылетела из могил",
    "Я работаю в крупная копмании",
    "Он часто бфвает на море"
]

russian_true = [
    "Я купил билет на рынке",
    "Мы пойдем на бал",
    "Она выглядит очень хорошо",
    "Вчера был тяжeлый день на работе",
    "Он учится в университете",
    "Это требует времени",
    "Я встречаюс с ней ужe два нода",
    "Мы собираемса на море в июне",
    "Пулец вылетела из могил",
    "Я работаю в крупная копмании",
    "Он часто бфвает на море"
]


* Without keyboard layout (Norvig's solution)

In [219]:
for sent, true in zip(russian_test, russian_true):
    sent = sent.lower()
    true = true.lower()
    correction = sent_correction(ru_sc, sent, with_keyboard_layout=False)
    if correction == true:
        print(f"{correction} == {true}")
    else:
        print(f"{correction} != {true}")

я купил билет на рынке == я купил билет на рынке
мы пойдет на бали != мы пойдем на бал
она выглядит очень хорошо == она выглядит очень хорошо


вчера был тяжелый цены на работе != вчера был тяжeлый день на работе
он учился в университет != он учится в университете
это требует времени == это требует времени
я встреча с нет уже два года != я встречаюс с ней ужe два нода
мы собираемся в морем в июне != мы собираемса на море в июне
пулей вылетела из могилы != пулец вылетела из могил
я работаю в крупные компании != я работаю в крупная копмании
он часто бывает на море != он часто бфвает на море


The result: russian corrector without keyboard layout corrected 3 out of 11 sentences.

* With keyboard layout

In [220]:
for sent, true in zip(russian_test, russian_true):
    sent = sent.lower()
    true = true.lower()
    correction = sent_correction(ru_sc, sent, with_keyboard_layout=True)
    if correction == true:
        print(f"{correction} == {true}")
    else:
        print(f"{correction} != {true}")

я купил билет на рынке == я купил билет на рынке
мы пойдет на бал != мы пойдем на бал
она выглядит очень хорошо == она выглядит очень хорошо
вчера был тяжелый деньги на работе != вчера был тяжeлый день на работе
он учился в университет != он учится в университете
это требует времени == это требует времени
я встреча с нет уже два года != я встречаюс с ней ужe два нода
мы собираемся в морем в июне != мы собираемса на море в июне
пулей вылетела из могилы != пулец вылетела из могил
я работаю в крупных компаний != я работаю в крупная копмании
он часто бывает на море != он часто бфвает на море


The result: russian corrector with keyboard layout corrected 3 out of 11 sentences.

# Conclusion 


I explored Norvig's solution and have extended the algorithm to consider keyboard layout and correct Russian language (instead of English). As in Norvig's I have used the notion of edit distance (or Levenstein distance), but with an important advancement. Costs were weighted by the distance between letters on keyboard. For example, replacing "а" with "я" will cost less, than "а" with "э" as they are located in different parts of keyboard.

I have evaluated the algorithm on a limited set of Russian sentences. As the dataset was not large enough, the change in performance was slight. I consider that the dataset contain too little of n-grams for the arbitrary sentences in Russian to be corrected with high accuracy.

This version of the algorithm tends to be biased to spelling mistakes rather than to semantic ones. That is directly explained by the change introduced to the edit distance algorithm.